<a href="https://colab.research.google.com/github/DGMiles/MSMUPChem/blob/main/syed_CNTower_3Body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="pics/Toronto-04.png" width="300" align="right">

## **Example 3.1. Simulation of a projectile on Earth.**
---

We want to know the dynamics of a green apple ($m = 0.3$ kg) tossed horizontally at 10 cm/s speed from the top of the Toronto CN Tower (553 m) for the first 10 seconds.

In [2]:
# Import necessary libraries: NumPy for numerical calculations and Matplotlib for visualization.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Create a new figure and axis for plotting.
fig, ax = plt.subplots(figsize=(8, 8))

# Set the limits and labels for the plot.
ax.set(xlim=(-2, 2), ylim=(0, 600), xlabel='Position, meters', ylabel='Height, meters', title='Apple falling from CN tower')

# Define parameters of the problem.
T = 10.0  # Total time of simulation in seconds.
m = 0.3   # Mass of the green apple in kilograms.
g = 9.8   # Acceleration due to gravity in meters per second squared.
v0x = -0.1  # Initial horizontal velocity of the apple in meters per second.
H = 553.0   # Height of the CN Tower in meters.

# Define the time step and calculate the number of time steps needed.
dt = 0.05  # Time step size in seconds.
N = int(T / dt)  # Number of time steps.

# Create arrays to store velocity, position, and force information over time.
v = np.zeros((N+1, 2))  # Array to store velocity vectors.
r = np.zeros((N+1, 2))  # Array to store position vectors.
f = np.zeros((N+1, 2))  # Array to store force vectors.

# Set initial conditions for position and velocity.
r[0] = np.array([0.0, H])  # Initial position of the apple at the top of the CN Tower.
v[0] = np.array([-v0x, 0.0])  # Initial velocity with only horizontal component.

# The only force acting on the apple is gravity.
f[:] = np.array([0.0, -m * g])  # Constant gravitational force in the negative y-direction.

# Simulate the dynamics by updating position and velocity over time.
for n in range(N):
    v[n+1] = v[n] + f[n] / m * dt  # Update velocity using Newton's second law.
    r[n+1] = r[n] + v[n+1] * dt  # Update position using the updated velocity.

# Create a scatter plot to visualize the initial data point.
scat = ax.scatter(r[0, 0], r[0, 1], marker='o', c='g', s=200)

# Define an animation function to update the position of the apple over time.
def animate(i):
    scat.set_offsets(r[i])

# Create an animation object using Matplotlib's FuncAnimation.
ani = animation.FuncAnimation(fig, func=animate, frames=N)

# Save the animation as an HTML file, specifying the frame rate.
ani.save('CNtower.html', writer=animation.HTMLWriter(fps=1 // dt))

# Close the plot.
plt.close()


Let's visualize the dynamics using embedded HTML. It's interactive and you can play a movie step by step:

In [3]:
import shutil
import os

# Define the folder and file names.
frames_folder = 'CNtower_frames'
html_file = 'CNtower.html'

# Create a temporary directory for zipping both the folder and file.
tmp_dir = '/content/tmp_folder'
os.makedirs(tmp_dir, exist_ok=True)

# Copy the folder and file to the temporary directory.
shutil.copytree(frames_folder, os.path.join(tmp_dir, frames_folder))
shutil.copy(html_file, os.path.join(tmp_dir, html_file))

# Zip the temporary directory.
shutil.make_archive('/content/CNTower', 'zip', tmp_dir)

# Remove the temporary directory.
shutil.rmtree(tmp_dir)

# Provide the download link for the zip file.
from google.colab import files
files.download('/content/CNTower.zip')

## 1) MAKE SURE TO UNZIP FOLDER
## 2) Then double click the html file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<!-- Create an HTML cell with custom CSS to adjust the width and height -->
<div style="width: 800px; height: 600px;">
    <iframe src="/content/Untitled Folder/CNtower.html" width="100%" height="100%"></iframe>
</div>


---
When a closed system of particles are interacting through pairwise potentials, the force on each particle $i$ depends on its position with respect to every other particle $j$:

$$m_i\frac{d^2r_i(t)}{dt^2} = \sum_jF_{ij}(t) = -\sum_j\nabla_i{U(|r_{ij}(t)|)}$$

where $r_{ij} = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2}$ is the distance between particle $i$ and $j$, and $i,j \in (1,N)$.

### Example 3.2. Simulation of 3-body problem with Hooke's law:

We want to know the dynamics of 3 particles $m = 1$ kg connected to each other with invisible springs with $K_s = 5$ N/m, and $r_0 = 1$ m initially located at (0, 2), (2, 0) and (-1, 0) on the 2D plane for the first 10 seconds of their motion.

**Hint:**
The pairwise potential is (Hooke's Law): $$U(r_{ij}) = \frac{K_s}{2}(r_{ij} - r_0)^2$$

The negative gradient of the potential is a force from $j$-th upon $i$-th:

$$\mathbf{F_{ij}} = - \nabla_i{U(r_{ij})} = - K_s (r_{ij} - r_0) \nabla_i r_{ij} = - K_s (r_{ij} - r_0) \mathbf{r_{ij}} / r_{ij}$$


In [4]:
# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(6,6))
ax.set(xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), ylabel='meters', xlabel='meters', title='3-Body problem')

# parameters of the problem
T = 10. #s
m = 1.0 #kg
ks = 5 #N/m
r0 = 1. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem: first axis - time. second axis - particle's number. third - coordinate
v = np.zeros((N+1, 3, 2))
r = np.zeros((N+1, 3, 2))
f = np.zeros((N+1, 3, 2))

# initial conditions for 3 particles:
r[0,0] = np.array([0., 2.])
r[0,1] = np.array([2., 0.])
r[0,2] = np.array([-1., 0.])

def compute_forces(n):
    '''The function computes forces on each pearticle at time step n'''
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[n,i] - r[n,j]
                rij_abs = np.linalg.norm(rij)
                f[n, i] -= ks * (rij_abs - r0) * rij / rij_abs
## Run dynamics:
for n in range(N):
    compute_forces(n)
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing and animating
scat = ax.scatter(r[0,:,0], r[0,:,1], marker='o', c=['b', 'k', 'r'], s=1000)

def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, animate, frames=N)
plt.close()
## this function will create a lot of *.png files in a folder '3Body_frames'
ani.save('3body.html', writer=animation.HTMLWriter(fps= 1//dt))

Again, looking at the trajectory representation in real time:

In [5]:

# Define the folder and file names.
frames_folder = '3body_frames'
html_file = '3body.html'

# Create a temporary directory for zipping both the folder and file.
tmp_dir = '/content/tmp_folder'
os.makedirs(tmp_dir, exist_ok=True)

# Copy the folder and file to the temporary directory.
shutil.copytree(frames_folder, os.path.join(tmp_dir, frames_folder))
shutil.copy(html_file, os.path.join(tmp_dir, html_file))

# Zip the temporary directory.
shutil.make_archive('/content/3body', 'zip', tmp_dir)

# Remove the temporary directory.
shutil.rmtree(tmp_dir)

# Provide the download link for the zip file.
from google.colab import files
files.download('/content/3body.zip')

## 1) MAKE SURE TO UNZIP FOLDER
## 2) Then double click the html file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>